# Find best model

### Imports

In [1]:
import __main__ as main

from helpers.training_classifier import *
from helpers.feature_selection import select_features
from helpers.paths import Paths
from joblib import load, dump
from sklearn.model_selection import train_test_split
from datetime import datetime
from helpers.is_interactive import is_interactive

### Run dependency notebooks

In [2]:
if is_interactive(main):
    %run 01_2_data_wrangling_classifier.ipynb -p
    %run 02_2_scaling_classifier.ipynb -p

Running previous notebooks...


### Load Dataframe

In [3]:
training_data = load(Paths.CLASSIFIER_SCALING_DATA)

df = training_data['dataset']

X = df.drop('type_unified', axis=1)
y = df['type_unified']

df.head(10)

Living space  Plot area     Floor  Latitude  Longitude       Zip  \
0      0.010330   0.002748  0.363636  0.808677   0.477811  0.464657   
1      0.016114   0.000633  0.314570  0.808677   0.477811  0.464657   
2      0.009606   0.002575  0.272727  0.799258   0.468164  0.462000   
3      0.015907   0.001054  0.181818  0.808677   0.477811  0.464657   
4      0.014668   0.003276  0.181818  0.803051   0.470341  0.464541   
5      0.019626   0.003029  0.181818  0.801165   0.461133  0.464079   
6      0.012809   0.000570  0.302524  0.808677   0.477811  0.464657   
7      0.014045   0.001995  0.318182  0.794885   0.467948  0.462000   
8      0.007747   0.000353  0.271420  0.801046   0.474388  0.465696   
9      0.011362   0.006030  0.181818  0.798179   0.461295  0.464079   

   distanceToTrainStation  gde_area_agriculture_percentage  \
0                0.114461                         0.349443   
1                0.114461                         0.349443   
2                0.034049                         0.129342   
3                0.114461                         0.349443   
4                0.054848                         0.377475   
5                0.118598                         0.375855   
6                0.114461                         0.349443   
7                0.027599                         0.129342   
8                0.038177                         0.129342   
9                0.106152                         0.375855   

   gde_area_forest_percentage  gde_area_nonproductive_percentage  ...  \
0                    0.586349                           0.051803  ...   
1                    0.586349                           0.051803  ...   
2                    0.366948                           0.080560  ...   
3                    0.586349                           0.051803  ...   
4                    0.566478                           0.013291  ...   
5                    0.606279                           0.013751  ...   
6                    0.586349                           0.051803  ...   
7                    0.366948                           0.080560  ...   
8                    0.366948                           0.080560  ...   
9                    0.606279                           0.013751  ...   

   WorkplaceDensity_2  WorkplaceDensity_3  WorkplaceDensity_4  \
0            0.004779            0.000330            0.000023   
1            0.004779            0.000330            0.000023   
2            0.036712            0.007034            0.001348   
3            0.004779            0.000330            0.000023   
4            0.002871            0.000154            0.000008   
5            0.006233            0.000492            0.000039   
6            0.004779            0.000330            0.000023   
7            0.120739            0.041954            0.014578   
8            0.010812            0.001124            0.000117   
9            0.005998            0.000465            0.000036   

   WorkplaceDensity_5  WorkplaceDensity_6  ForestDensity_2  ForestDensity_3  \
0        1.578986e-06        1.091576e-07         0.094108         0.028869   
1        1.578986e-06        1.091576e-07         0.094108         0.028869   
2        2.582432e-04        4.948057e-05         0.008129         0.000733   
3        1.578986e-06        1.091576e-07         0.094108         0.028869   
4        4.415364e-07        2.365700e-08         0.068665         0.017993   
5        3.067084e-06        2.421424e-07         0.017961         0.002407   
6        1.578986e-06        1.091576e-07         0.094108         0.028869   
7        5.065411e-03        1.760101e-03         0.000314         0.000006   
8        1.215577e-05        1.263976e-06         0.129677         0.046698   
9        2.786040e-06        2.157669e-07         0.022775         0.003437   

   ForestDensity_4  ForestDensity_5  ForestDensity_6  
0     8.856293e-03     2.716846e-03     8.334470e-04  
1     8.856293e-03     2.716846e-03   

## Train test split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

## Train all Models

In [5]:
TRAINING_FUNCTIONS = [
    train_k_neighbors,
    train_gradient_boosting,
    train_random_forest,
    train_mlp_classifier
]

In [6]:
results = [
    func(X_train, X_test, y_train, y_test)
    for func in TRAINING_FUNCTIONS
]

Training KNeighborsClassifier with -1 jobs
Parameters: {'n_neighbors': [5], 'weights': ['distance'], 'leaf_size': [10], 'p': [2]}
Training GradientBoostingClassifier with -1 jobs
Parameters: {'max_depth': [2], 'min_samples_split': [10], 'max_features': [0.5], 'min_samples_leaf': [10], 'n_estimators': [130], 'random_state': [42]}
Training RandomForestClassifier with -1 jobs
Parameters: {'min_samples_split': [10], 'max_features': [0.5], 'min_samples_leaf': [10], 'n_estimators': [130], 'random_state': [42]}
Training MLPClassifier with 1 jobs
Parameters: {'hidden_layer_sizes': [(14, 14, 14, 14)], 'activation': ['relu'], 'learning_rate': ['adaptive'], 'learning_rate_init': [0.005], 'max_iter': [10000]}


In [12]:
pd.DataFrame(results)

num_columns     score                                        best_params  \
0          222  0.633226  {'n_neighbors': 5, 'weights': 'distance', 'lea...   
1          222  0.713184  {'max_depth': 2, 'min_samples_split': 10, 'max...   
2          222  0.762381  {'min_samples_split': 10, 'max_features': 0.5,...   
3          222  0.649037  {'hidden_layer_sizes': (14, 14, 14, 14), 'acti...   

                                               model  
0  KNeighborsClassifier(leaf_size=10, weights='di...  
1  ([DecisionTreeRegressor(criterion='friedman_ms...  
2  (DecisionTreeClassifier(max_features=0.5, min_...  
3  MLPClassifier(hidden_layer_sizes=(14, 14, 14, ...

## Select the best model

In [13]:
best_model_row = sorted(results, key=lambda x: x['score'])[-1]
best_model = best_model_row['model']
best_model

RandomForestClassifier(max_features=0.5, min_samples_leaf=10,
                       min_samples_split=10, n_estimators=130, random_state=42)

In [16]:
best_model_row['best_params']

{'min_samples_split': 10,
 'max_features': 0.5,
 'min_samples_leaf': 10,
 'n_estimators': 130,
 'random_state': 42}

## Save Results

In [17]:
def getClassName(obj):
   return type(obj).__name__

def getFormattedDate(date = datetime.now()):
    return date.strftime('%Y%m%d_%H%M')

In [18]:
for result in results:
    dump(
        result['model'],
        Paths.CLASSIFIER_MODEL_DATA(getClassName(result['model']))
    )